In [1]:
import torch 
from Models.fader_network import FaderNetwork
import numpy as np
from Data.preprocess import load_images

In [2]:
# Configuration des paramètres
selected_attrs = ['Smiling', 'Male', 'Eyeglasses', 'Young', 'Mouth_Slightly_Open']
num_attributes = len(selected_attrs)
end = 202599
start = 200000
index = (start, end)
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Utilisation du device : {device}")

auto_encoder_full_BD_100_epochs = FaderNetwork(attribute_dim=num_attributes, attributes=selected_attrs).to(device)
auto_encoder_batch_60K_100_epochs = FaderNetwork(attribute_dim=num_attributes, attributes=selected_attrs).to(device)
auto_encoder_batch_50K_50_epochs = FaderNetwork(attribute_dim=num_attributes, attributes=selected_attrs).to(device)

(train_loader, valid_loader, test_loader) = load_images(index = index, selected_attrs = selected_attrs, batch_size=32)

# On charge le classifier du GitHub de l'article pour évaluer le modèle 
torch.nn.Module.dump_patches = True
clf = torch.load('./classifier256.pth', map_location=torch.device(device), weights_only=False)
clf.eval()

# On charge le modèle qui à appris sur toute le base de données avec 100 epochs
auto_encoder_full_BD_100_epochs.load_state_dict(torch.load('./Models/trained_model/TrainBD_100_full/auto_encoder_epoch_100.pth'))
auto_encoder_full_BD_100_epochs.eval()

# On charge le modèle qui à appris sur toute le base de données mais avec un batch_size de 60K images et 100 epochs
auto_encoder_batch_60K_100_epochs.load_state_dict(torch.load('./Models/trained_model/TrainBD_50_100_2/auto_encoder_epoch_100.pth'))
auto_encoder_batch_60K_100_epochs.eval()

# On charge le modèle qui à appris sur toute le base de données mais avec un batch_size de 50K images et 50 epochs
auto_encoder_batch_50K_50_epochs.load_state_dict(torch.load('./Models/trained_model/TrainBD_50_full_4//auto_encoder_epoch_50.pth'))
auto_encoder_batch_50K_50_epochs.eval()

print("Modèles chargés et prêts pour le test.")

Utilisation du device : cuda


Processing Attribute: 100%|██████████| 2597/2597 [00:00<00:00, 43114.45it/s]
/usr/local/lib/python3.10/dist-packages/torch/serialization.py:868: SourceChangeWarning: source code of class 'torch.nn.modules.container.Sequential' has changed. Saved a reverse patch to Sequential.patch. Run `patch -p0 < Sequential.patch` to revert your changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.10/dist-packages/torch/serialization.py:868: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. Tried to save a patch, but couldn't create a writable file Conv2d.patch. Make sure it doesn't exist and your working directory is writable.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.10/dist-packages/torch/serialization.py:868: SourceChangeWarning: source code of class 'torch.nn.modules.activation.LeakyReLU' has changed. Saved a reverse patch to LeakyReLU.patch. Run `patch -p0 < LeakyReLU.patch` to revert your changes.
  warnings.warn(m

Modèles chargés et prêts pour le test.


In [3]:
def accuracy_classifier(output, attributes, att_position):
    """
    Calcule la précision pour les prédictions du discriminateur latent.
    Args:
        output (torch.Tensor): Les prédictions du modèle (batch_size, 2*n_attributes).
        attributes (torch.Tensor): Les attributs cibles (batch_size, 2*n_attributes).
    Returns:
        float: Précision totale sur le batch.
    """
    att_accuracies = []
    accuracy = 0 # Initialisation de la précision totale
    n_cat = 2 # Chaque attribut est encodé sur deux colonnes (codage one hot)
    for k, p in zip(range(0, attributes.size(1), n_cat), att_position):
        # Sélectionner les prédictions pour l'attribut courant
        pred = output[:, p:p + n_cat].argmax(1)
        # Obtenir les labels cibles pour cet attribut
        y = attributes[:, k:k + n_cat].argmax(1)
        # Vérification des correspondances entre prédictions et cibles
        accuracy = pred.eq(y.view_as(pred)).sum().item()
        att_accuracies.append(accuracy)
    return att_accuracies

In [4]:
att_position = [62, 40, 30, 78, 42]
att_position_start = [-1, 0, 2, 4, 6, 8]
model_clf_acc_real_img = np.zeros(num_attributes)
model_clf_acc_gen_img_model_full_bd = np.zeros(num_attributes)
model_clf_acc_gen_img_model_60_bd = np.zeros(num_attributes)
model_clf_acc_gen_img_model_50_bd = np.zeros(num_attributes)

with torch.no_grad():
    for l, (X, y) in enumerate(test_loader):
        X, y = X.to(device), y.to(device)
        flip_start = np.random.choice(att_position_start)
        if flip_start == -1:
            target_gen = 1 - y
        else:
            target_gen = y.clone()
            target_gen[:, flip_start:flip_start+2] = 1 - target_gen[:, flip_start:flip_start+2]
            
        _, D1 = auto_encoder_full_BD_100_epochs(X, target_gen)
        _, D2 = auto_encoder_batch_60K_100_epochs(X, target_gen)
        _, D3 = auto_encoder_batch_50K_50_epochs(X, target_gen)
        
        clf_outputs_1 = clf(D1)
        clf_outputs_2 = clf(D2)
        clf_outputs_3 = clf(D3)
        clf_outputs_real_image = clf(X)
        
        acc_1 = accuracy_classifier(clf_outputs_1, target_gen, att_position)
        acc_2 = accuracy_classifier(clf_outputs_2, target_gen, att_position)
        acc_3 = accuracy_classifier(clf_outputs_3, target_gen, att_position)
        acc_real_img = accuracy_classifier(clf_outputs_real_image, y, att_position)
        
        model_clf_acc_gen_img_model_full_bd += np.array(acc_1)
        model_clf_acc_gen_img_model_60_bd += np.array(acc_2)
        model_clf_acc_gen_img_model_50_bd += np.array(acc_3)
        model_clf_acc_real_img += np.array(acc_real_img)
        
    model_clf_acc_gen_img_model_full_bd /= len(test_loader.dataset)
    model_clf_acc_gen_img_model_60_bd /= len(test_loader.dataset)
    model_clf_acc_gen_img_model_50_bd /= len(test_loader.dataset)
    model_clf_acc_real_img /= len(test_loader.dataset)

In [5]:
print("Performance du classifieur sur les Images Réelles")
for name, prob in zip(selected_attrs, model_clf_acc_real_img):
    print(f'Attribut {name} accuracy : {prob*100} % \n')

Performance du classifieur sur les Images Réelles
Attribut Smiling accuracy : 93.53846153846153 % 

Attribut Male accuracy : 97.84615384615385 % 

Attribut Eyeglasses accuracy : 99.84615384615385 % 

Attribut Young accuracy : 88.61538461538461 % 

Attribut Mouth_Slightly_Open accuracy : 93.84615384615384 % 



In [6]:
print("Performance du classifieur sur les Images générées avec le modèle auto_encoder_full_BD_100_epochs")
for name, prob in zip(selected_attrs, model_clf_acc_gen_img_model_full_bd):
    print(f'Attribut {name} accuracy : {prob*100} % \n')

Performance du classifieur sur les Images générées avec le modèle auto_encoder_full_BD_100_epochs
Attribut Smiling accuracy : 94.0 % 

Attribut Male accuracy : 88.61538461538461 % 

Attribut Eyeglasses accuracy : 72.92307692307692 % 

Attribut Young accuracy : 72.92307692307692 % 

Attribut Mouth_Slightly_Open accuracy : 83.07692307692308 % 



In [7]:
print("Performance du classifieur sur les Images générées avec le modèle auto_encoder_batch_60K_100_epochs")
for name, prob in zip(selected_attrs, model_clf_acc_gen_img_model_60_bd):
    print(f'Attribut {name} accuracy : {prob*100} % \n')

Performance du classifieur sur les Images générées avec le modèle auto_encoder_batch_60K_100_epochs
Attribut Smiling accuracy : 86.76923076923076 % 

Attribut Male accuracy : 89.38461538461539 % 

Attribut Eyeglasses accuracy : 74.92307692307692 % 

Attribut Young accuracy : 73.6923076923077 % 

Attribut Mouth_Slightly_Open accuracy : 76.76923076923077 % 



In [8]:
print("Performance du classifieur sur les Images générées avec le modèle auto_encoder_batch_50K_50_epochs")
for name, prob in zip(selected_attrs, model_clf_acc_gen_img_model_50_bd):
    print(f'Attribut {name} accuracy : {prob*100} % \n')

Performance du classifieur sur les Images générées avec le modèle auto_encoder_batch_50K_50_epochs
Attribut Smiling accuracy : 91.84615384615384 % 

Attribut Male accuracy : 92.61538461538461 % 

Attribut Eyeglasses accuracy : 91.07692307692308 % 

Attribut Young accuracy : 71.07692307692308 % 

Attribut Mouth_Slightly_Open accuracy : 84.15384615384616 % 

